In [1]:
import os

In [2]:
%pwd

'c:\\Users\\USER\\Documents\\JupyterNB\\ineuron\\Python Advanced\\Sentiment_Analysis_Project\\research'

In [3]:
os.chdir('../')
%pwd

'c:\\Users\\USER\\Documents\\JupyterNB\\ineuron\\Python Advanced\\Sentiment_Analysis_Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelBuildingConfig:
    root_dir: Path
    base_modl_file: Path
    trained_modl_file: Path
    test_data_file: Path
    number_of_neurons_in_first_layer: int
    number_of_neurons_in_second_layer: int
    number_of_neurons_in_output_layer: int
    metrics: str
    learning_rate: float
    epochs: int

    



In [5]:
from src.sentimentAnalysis.constants import *
from src.sentimentAnalysis.utils.common_functionality import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_building_config(self) -> ModelBuildingConfig:
        config = self.config.model_building

        create_directories([config.root_dir])

        model_building_config = ModelBuildingConfig(
            root_dir = config.root_dir,
            base_modl_file = config.base_modl_file,
            trained_modl_file = config.trained_modl_file,
            test_data_file = config.test_data_file,
            number_of_neurons_in_first_layer = self.params.NUMBER_OF_NEURONS_IN_FIRST_LAYER,
            number_of_neurons_in_second_layer = self.params.NUMBER_OF_NEURONS_IN_SECOND_LAYER,
            number_of_neurons_in_output_layer = self.params.NUMBER_OF_NEURONS_IN_OUTPUT_LAYER,
            metrics = self.params.METRICS,
            learning_rate = self.params.LEARNING_RATE,
            epochs = self.params.EPOCHS

            
        )

        return model_building_config

In [7]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from src.sentimentAnalysis.utils.common_functionality import load_object, save_object, get_size
from src.sentimentAnalysis import logger

In [8]:
class ModelBuilding:
    def __init__(self, config: ModelBuildingConfig):
        self.config = config


    
    def prepare_base_model(self):
        if not os.path.exists(self.config.base_modl_file):
            model = Sequential()

            model.add(LSTM(self.config.number_of_neurons_in_first_layer, input_shape = (1,100), return_sequences = True))
            model.add(LSTM(self.config.number_of_neurons_in_second_layer))
            model.add(Dense(self.config.number_of_neurons_in_output_layer, activation = 'sigmoid'))

            print(model.summary())

            loss = keras.losses.BinaryCrossentropy(from_logits = True)
            opt = keras.optimizers.Adam(learning_rate=self.config.learning_rate)
            metrices = [self.config.metrics]

            model.compile(loss=loss,optimizer=opt,metrics=metrices)
            logger.info("Model comiplation is done")

            save_object(self.config.base_modl_file,model)
            logger.info("Base model is saved")


        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def train_model(self,final_data_path):
        if not os.path.exists(self.config.trained_modl_file):
            
            final_data_for_training = load_object(final_data_path)

            X = final_data_for_training['X']
            y = final_data_for_training['y']

            X = X.reshape(X.shape[0],1,X.shape[1])
            y = y.reshape(-1,1)

            X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

            base_model = load_object(self.config.base_modl_file)

            base_model.fit(x=X_train,y=y_train,epochs=self.config.epochs)
            logger.info("Model training completed")

            save_object(self.config.trained_modl_file, base_model)
            logger.info("trained model is saved")

            test_data = {
                'X_test': X_test,
                'y_test': y_test
            }

            save_object(self.config.test_data_file, test_data)
            logger.info("test data saved for evaluation phase")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            


In [9]:
try:
    config = ConfigurationManager()
    
    model_building_config = config.get_model_building_config()
    model_building = ModelBuilding(config=model_building_config)
    model_building.prepare_base_model()
    model_building.train_model(config.config.data_processing.final_data_file)
   
except Exception as e:
    raise e

[2023-07-13 09:22:28,338: INFO: common_functionality: yaml file: config\config.yaml loaded successfully]
[2023-07-13 09:22:28,406: INFO: common_functionality: yaml file: params.yaml loaded successfully]
[2023-07-13 09:22:28,415: INFO: common_functionality: created directory at: artifacts]
[2023-07-13 09:22:28,433: INFO: common_functionality: created directory at: artifacts/model_building]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 128)            117248    
                                                                 
 lstm_1 (LSTM)               (None, 256)               394240    
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 511745 (1.95 MB)
Trainable params: 511745 (1.9

c:\Users\USER\anaconda3\envs\sentimentenv\lib\site-packages\keras\src\backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1240/1240 [==============================] - 25s 14ms/step - loss: 0.3536 - accuracy: 0.8469
Epoch 2/7
1240/1240 [==============================] - 17s 14ms/step - loss: 0.3292 - accuracy: 0.8574
Epoch 3/7
1240/1240 [==============================] - 17s 14ms/step - loss: 0.3226 - accuracy: 0.8615
Epoch 4/7
1240/1240 [==============================] - 17s 14ms/step - loss: 0.3170 - accuracy: 0.8635
Epoch 5/7
1240/1240 [==============================] - 17s 14ms/step - loss: 0.3121 - accuracy: 0.8642
Epoch 6/7
1240/1240 [==============================] - 17s 14ms/step - loss: 0.3073 - accuracy: 0.8670
Epoch 7/7
1240/1240 [==============================] - 17s 14ms/step - loss: 0.3017 - accuracy: 0.8700
[2023-07-13 09:24:42,425: INFO: 3232696556: Model training completed]
[2023-07-13 09:24:42,751: INFO: 3232696556: trained model is saved]
